# IOT-IPS

The script for iot-ips project

Current use a simple fully-connected regressor with averaged rssi + csi information.

If you want to add more information to input, you can modify the parser function.

In [1]:
import os
import sys
import csv
import math
import torch
import numpy as np
import torch.nn as nn
import pandas as pd

from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader, random_split

/home/rtu/anaconda3/envs/habitat/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
myseed = 6666  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Data

## Define the input format

In [3]:
def parser(rssi, csi, position):
    '''
    description:
        this is a function to rearrange the raw data into the form of your model input
        so that you can easily design the input format and arrange the input data.
        
    input:
        rssi: the averaged rssi in one collect file, stored in numpy array
        csi: an array consists of all csi data in one collect file, stored in a 2D numpy array
        position: nparray(x,y)
        Note: both the rssi and csi belong to the same location
        
    output:
        x: the list containing inputs of the model
        y: the correspond ground truth
    '''
    
    signal = np.concatenate((rssi,csi.mean(axis=0)))
    x = [signal,]
    y = [position,]
    
    return x, y # use rssi data only as example

## Dataset

In [4]:
class Dataset(Dataset):
    
    def __init__(self, path, parser):
        filenames = os.listdir(path)
        self.parser = parser
        
        self.x = []
        self.y = []
        
        for name in filenames:
            if 'csi' in name:
                file = os.path.join(path, name)
                x,y = self.readOne(file)
                self.x.extend(x)
                self.y.extend(y)
                      
    def readOne(self, fname):
        
        if 'csi' in fname:
            csi = fname
            rssi = fname.replace('csi','rssi')
        elif 'rssi' in fname:
            rssi = fname
            csi = fname.replace('rssi','csi')
        
        # parse the position from the file name
        position = csi[csi.find('csi')+3: csi.rfind('.')].split('_')
        position = np.asfarray([float(i) for i in position])
        
        rssi_f = open(rssi)
        csi_f = open(csi)
        rssilines = rssi_f.readlines()
        csilines = csi_f.readlines()
        
        # Read the rssi and average
        row = 0
        rssi_data = np.zeros(8, dtype=float)
        for line in rssilines:
            rssi_raw = line.split(',')
            try:
                rssi_data += np.asfarray([int(i) for i in rssi_raw])
                row += 1
            except:
                continue
        rssi_data /= row
            
        # Read the csi
        csi_data = []
        for line in csilines:
            csi_raw = line.split(',')[-2][1:-2]  # skip the brackets and space
            csi_str = csi_raw.split(' ')
            csi_float = np.asfarray([int(i) for i in csi_str])
            csi_data.append(csi_float)
        csi_data = np.asfarray(csi_data)
        
        rssi_f.close()
        csi_f.close()
        
        x,y = self.parser(rssi_data, csi_data, position)
        x = [torch.Tensor(i) for i in x]
        y = [torch.Tensor(i) for i in y]
        
        return x,y
        
    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

# Model

In [5]:
class Model(nn.Module):
    def __init__(self, input_dim=136, hidden=32):
        super(Model, self).__init__()
        
        ylayers = [
            nn.Linear(input_dim,hidden),
            nn.LeakyReLU(),
            nn.Linear(hidden,1),
        ]
        
        xlayers = [
            nn.Linear(input_dim,hidden),
            nn.LeakyReLU(),
            nn.Linear(hidden,1),
        ]
        
        self.xlayers = nn.Sequential(*xlayers)
        self.ylayers = nn.Sequential(*ylayers)
        
    def forward(self, s):
        x = self.xlayers(s)
        y = self.ylayers(s)
        o = torch.concat((x,y), axis=1)
        return o

# Hyper Parameters

In [6]:
lr = 0.0001
momentum = 0.01
n_epochs = 10000
batch_size = 32
valid_epoch = 1000
save_path = './test.pt'
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [7]:
train_dataset = Dataset('data/train', parser)
test_dataset = Dataset('data/test', parser)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)
valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)

model = Model().to(device)

In [8]:
criterion = nn.MSELoss(reduction='mean')
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum) 


best_loss, step, early_stop_count = math.inf, 0, 0

train_pbar = tqdm(range(n_epochs), position=0, leave=True)
for epoch in train_pbar:
    model.train()
    loss_record = []

    for x, y in train_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x+torch.rand(x.size()).to(device))  
        loss = criterion(pred, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        step += 1
        loss_record.append(loss.detach().item())

        # Display current epoch number and loss on tqdm progress bar.
        train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
        train_pbar.set_postfix({'loss': loss.detach().item()})

    if epoch % valid_epoch==0:
        mean_train_loss = sum(loss_record)/len(loss_record)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

            loss_record.append(loss.item())

        mean_valid_loss = sum(loss_record)/len(loss_record)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f}')

    if mean_valid_loss < best_loss:
        best_loss = mean_valid_loss
        torch.save(model.state_dict(), save_path)
        print('Saving model with loss {:.3f}...'.format(best_loss))


Epoch [50/10000]:   0%|           | 49/10000 [00:00<00:41, 239.91it/s, loss=3.6]

Epoch [1/10000]: Train loss: 8.8263, Valid loss: 13.7398
Saving model with loss 13.740...


Epoch [1046/10000]:  10%|▋     | 1043/10000 [00:04<00:37, 241.16it/s, loss=2.19]

Epoch [1001/10000]: Train loss: 2.1831, Valid loss: 5.4317
Saving model with loss 5.432...


Epoch [2049/10000]:  20%|█▏    | 2041/10000 [00:08<00:30, 259.37it/s, loss=2.06]

...

Epoch [8050/10000]:  80%|████▊ | 8036/10000 [00:33<00:08, 241.68it/s, loss=1.48]

Epoch [8001/10000]: Train loss: 1.4398, Valid loss: 4.3774


Epoch [9062/10000]:  90%|█████▍| 9048/10000 [00:37<00:03, 275.52it/s, loss=1.27]

Epoch [9001/10000]: Train loss: 1.2730, Valid loss: 4.6047


Epoch [10000/10000]: 100%|████| 10000/10000 [00:41<00:00, 242.97it/s, loss=1.28]


In [9]:
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, pin_memory=True)

model.eval() # Set your model to evaluation mode.
loss_record = []
with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        print(f'Input: {x.cpu().detach().numpy()[0]}')
        print(f'Prediction: {pred.cpu().detach().numpy()[0]}')
        print(f'GT        : {y.cpu().detach().numpy()[0]}\n\n')

Input: [-2.70e+01 -3.30e+01 -4.50e+01 -3.60e+01 -4.10e+01 -4.50e+01 -5.50e+01
 -4.50e+01  0.00e+00  0.00e+00 -1.06e+00  5.60e-01 -1.68e+00  1.80e-01
 -1.52e+00  4.40e-01 -1.28e+00  1.40e-01 -1.28e+00  1.20e-01 -1.28e+00
  4.80e-01 -1.14e+00  2.60e-01 -1.08e+00  2.00e-01 -1.14e+00  0.00e+00
 -1.24e+00  2.20e-01 -1.18e+00 -2.00e-02 -1.10e+00 -4.00e-02 -1.20e+00
  0.00e+00 -1.16e+00  2.00e-01 -1.14e+00  9.60e-01 -1.14e+00  1.40e-01
 -1.06e+00 -4.00e-02 -9.00e-01 -4.00e-02 -1.10e+00 -3.20e-01 -8.00e-01
 -4.00e-01 -1.22e+00 -2.40e-01 -1.08e+00 -3.60e-01 -9.40e-01 -4.00e-01
 -8.80e-01 -4.20e-01 -9.00e-01 -4.60e-01 -8.60e-01 -8.60e-01  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00  0.00e+00
 -1.56e+00  4.20e-01 -1.64e+00  3.00e-01 -1.66e+00  1.20e-01 -1.62e+00
  1.80e-01 -1.70e+00  1.80e-01 -1.70e+00  1.80e-01 -1.92e+00  3.00e-01